In [15]:
import dataloader
import madmom
import utilities
import features
import librosa
import scipy
import matplotlib.pyplot as plt
import numpy as np

def harmonicPercussiveResidualSeparation(sig,beta=2,n_fft=1024,fs=22050):
    time_filter_length = 0.2 # seconds
    frequency_filter_length = 500 # Hz
    hs = n_fft // 2 # hopsize
    X = librosa.stft(sig,n_fft=n_fft,hop_length=n_fft//2)
    Y = np.abs(X) 
    L_med_f = int(500 / (fs/n))
    L_med_t = int(0.2 / (hs /fs))
    Y_p = scipy.ndimage.median_filter(Y, (L_med_f,1))
    Y_h = scipy.ndimage.median_filter(Y, (1,L_med_t))

    M_h = Y_h / (Y_p+np.finfo(float).eps) > beta
    M_p = Y_p / (Y_h+np.finfo(float).eps) >= beta
    M_r = np.ones_like(M_h) ^ (M_h + M_p)

    X_h = M_h * X
    X_p = M_p * X
    X_r = M_r * X

    y_harm = librosa.istft(X_h,n_fft=n,hop_length=hs)
    y_perc = librosa.istft(X_p,n_fft=n,hop_length=hs)
    y_res = librosa.istft(X_r,n_fft=n,hop_length=hs)
    return y_harm,y_perc,y_res
    
model_path = [f"/home/max/ET-TI/Masterarbeit/models/ismir2016/chroma_dnn_3.pkl"]
dcp = madmom.audio.chroma.DeepChromaProcessor(fmin=30, fmax=5500, unique_filters=False,models=model_path)
dataset = dataloader.MIRDataset("beatles",basepath="/home/max/ET-TI/Masterarbeit/mirdata/",split_nr=3)
audiopath,gt = dataset["0105"]  # 1206 letitbe, 1208 i've got a feeling,girl  0609, 0304

In [67]:
# DEMO violin+drums
t_vec, sig_violin = utilities.loadAudio("/home/max/ET-TI/Masterarbeit/datasets/violin.wav")
t_vec, sig_drums = utilities.loadAudio("/home/max/ET-TI/Masterarbeit/datasets/drums.wav")
sig = sig_violin+sig_drums
y_harm,y_perc,y_res = harmonicPercussiveResidualSeparation(sig,beta=2,n_fft=4096)
_,y_perc,_ = harmonicPercussiveResidualSeparation(y_perc+y_res,beta=2,n_fft=512)

y_harm = y_harm/np.max(y_harm)
y_perc = y_perc/np.max(y_perc)
scipy.io.wavfile.write("/home/max/ET-TI/Masterarbeit/datasets/harmonic_example.wav", 22050, y_harm.astype(np.float32))
scipy.io.wavfile.write("/home/max/ET-TI/Masterarbeit/datasets/percussive_example.wav", 22050, y_perc.astype(np.float32))
